## Model description

This is system models a heater with a thermostat controller. Initially, the temperature is $\textrm{temperature} = 20^\circ C$ and the heater is on. The controller keeps the temperature between $16^\circ C$ and $24^\circ C$.  The heater is switched off if the temperature rises above $23^\circ C$ and it is switched on at a
temperature below $18^\circ C$.

Adding a controller to the model introduces new variables for the low and high temperature sensors in the controller, a variable for the actuator (heater) state in the plant and the controller, and a variable to store the controller mode. Additionally, we introduce a new clock for the cycle time of the PLC. The global time horizon is 10 seconds and the PLC cycle time is 0.5 seconds.

The dynamics of the temperature is defined by the following differential equations:

- $T'(t) = -0.1T(t) + 3$ if the heater is `on`
- $T'(t) = -0.1T(t)$ if the heater is `off`

This model has 8 continuous variables, 8 modes and 18 discrete jumps.

The model parameters and description are taken from the [HyPro collection of continuous and hybrid system benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/benchmarks-of-continuous-and-hybrid-systems/), see the [Thermostat with controller](https://ths.rwth-aachen.de/research/projects/hypro/thermostat-with-controller/).

---

*References:*

[1] J. Nellen. Analysis and synthesis of hybrid systems in engineering applications. PhD Thesis, RWTH Aachen University, 2016.

[2] S. Schupp et al.. Divide and conquer: Variable set separation in hybrid systems reachability analysis. Submitted to (QAPL’17).

In [ ]:
using Revise # to debug
using Reachability, HybridSystems, MathematicalSystems, LazySets, LinearAlgebra, SX, SymEngine
using Plots, LaTeXStrings

In [ ]:
#=
Gives:

read 2 components, but models with more than one component are not yet implemented; try flattening the model

we can possibly ignore the "system" components

file = "5_dim_linear_switch.xml"
readsxmodel(file, raw_dict=true)

=#

In [ ]:
# here we have manually removed the "system" component
file = "thermostat_spaceEx_separateControllerAndPlant_with_timer_flat.xml"
model = readsxmodel(file, raw_dict=true)

In [ ]:
model["flows"][1]

In [ ]:
function linear_switching(; X0 = Singleton([3.1, 4.0, 0.0, 0.0, 0.0]),
                            U = Interval(-1.0, 1.0),
                            T = 1.0,
                            ε = 1e-6)
        
    id_location = 1
    n = 5 # number of state variables
    m = 1 # number of input variables
    state_vars = convert.(Basic, [fi.args[1].args[1] for fi in model["flows"][id_location]])
    input_vars = [convert(Basic, collect(keys(model["variables"]))[6])] # same as convert(Basic, :u)

    A, B, c = get_coeffs(model["flows"][id_location], n, m, state_vars, input_vars) # new function in SX
    X = Universe(n)
    U = Interval(-1.0, 1.0)
    q1 = ConstrainedLinearControlContinuousSystem(A, B, X, U)

    id_location = 2
    A, B, c = get_coeffs(model["flows"][id_location], n, m, state_vars, input_vars)
    X = Universe(n)
    U = Interval(-1.0, 1.0)
    q2 = ConstrainedLinearControlContinuousSystem(A, B, X, U)
        
    id_location = 3
    A, B, c = get_coeffs(model["flows"][id_location], n, m, state_vars, input_vars)
    X = Universe(n)
    U = Interval(-1.0, 1.0)
    q3 = ConstrainedLinearControlContinuousSystem(A, B, X, U)  

    id_location = 4
    A, B, c = get_coeffs(model["flows"][id_location], n, m, state_vars, input_vars) # new function in SX
    X = Universe(n)
    U = Interval(-1.0, 1.0)
    q4 = ConstrainedLinearControlContinuousSystem(A, B, X, U)

    id_location = 5
    A, B, c = get_coeffs(model["flows"][id_location], n, m, state_vars, input_vars) # new function in SX
    X = Universe(n)
    U = Interval(-1.0, 1.0)
    q5 = ConstrainedLinearControlContinuousSystem(A, B, X, U)
        

    # automaton structure
    automaton = LightAutomaton(5)

    modes = [q1, q2, q3, q4, q5]

    # transitions
    add_transition!(automaton, 1, 2, 1)
    add_transition!(automaton, 2, 3, 2)
    add_transition!(automaton, 3, 4, 3)
    add_transition!(automaton, 4, 5, 4)
    add_transition!(automaton, 5, 1, 5)

    # guards
    G12 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0, 0.0, 0.0], 3.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0, 0.0, 0.0], -3.0 + ε)]) # x1 = 3

    G23 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0, 0.0, 0.0], 2.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0, 0.0, 0.0], -2.0 + ε)]) # x1 = 2

    G34 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0, 0.0, 0.0], 1.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0, 0.0, 0.0], -1.0 + ε)]) # x1 = 1

    G45 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0, 0.0, 0.0], 0.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0, 0.0, 0.0], -0.0 + ε)]) # x1 = 0

    G51 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0, 0.0, 0.0], 1.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0, 0.0, 0.0], -1.0 + ε)]) # x1 = 1

    resetmaps = [ConstrainedIdentityMap(2, G12), 
                 ConstrainedIdentityMap(2, G23),
                 ConstrainedIdentityMap(2, G34),
                 ConstrainedIdentityMap(2, G45),
                 ConstrainedIdentityMap(2, G51)];

    # switching
    switchings = [AutonomousSwitching()]

    ℋ = HybridSystem(automaton, modes, resetmaps, switchings)

    # initial condition in mode q1
    initial_condition = [(1, X0)]

    problem = InitialValueProblem(ℋ, initial_condition)

    options = Options(:mode=>"reach", :T=>T, :plot_vars=>[1, 2], :project_reachset=>false)

    return problem, options
end

## Reachability settings

We consider an initial set of

$$
    T = 20 \\
    \text{low} = 1\\
    \text{high} = 0\\
    H = H\_{\text{plc}} = 1\\
    \text{nextSfcLoc} = 1\\
    \text{cycle_time} = 0\\
    \text{global_time} = 0
$$

and the starting location `clock__switch_on_in_comm`, a time horizon `global_time=10s`, a PLC cycle duration of $\delta = 0.5s$, and a time step $r=0.01s$.

In [ ]:
X0 = Singleton([3.1, 4.0, 0.0, 0.0, 0.0])
U = Interval(-1.0, 1.0)
problem, options = linear_switching(; X0=X0, U=U, T = 1.0, ε = 1e-6);

## Results

In [ ]:
using LazySets.Approximations
using LazySets.Approximations: project, overapproximate

@time begin
    opC = BFFPSV18(:δ=>0.005)
    opD = ApproximatingDiscretePost()
    sol = solve(problem, options, opC, opD)
end;

In [ ]:
nsamples = length(sol.Xk)

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 2], Hyperrectangle) for i in eachindex(sol.Xk)];
plot([Xkproj[i] for i in 1:1:220], xlab="x1", ylab="x2", alpha=.3)

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 3], Hyperrectangle) for i in eachindex(sol.Xk)];
plot([Xkproj[i] for i in 1:1:220], xlab="x1", ylab="x3", alpha=.3)

In [ ]:
Xkproj = [project(sol.Xk[i].X, [2, 3], Hyperrectangle) for i in eachindex(sol.Xk)];
plot([Xkproj[i] for i in 1:1:220], xlab="x2", ylab="x3", alpha=.3)

In [ ]:
using Polyhedra

# using Concrete Discrete Post
@time begin
    opC = BFFPSV18(:δ=>0.01)
    opD = ConcreteDiscretePost()
    sol = solve(problem, options, opC, opD) # does it produce 
end;

In [ ]:
# similar to approximating discrete post ... gives big overapprox error
# with default options
using LazySets.Approximations

@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = ApproximatingDiscretePost()
    sol = solve(RodReactor, options, opC, opD)
end;

In [ ]:
# similar to approximating discrete post ... gives big overapprox error
# with default options. we try to use oct direction but it doesn't use them
using LazySets.Approximations

@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = LazyDiscretePost(:check_invariant_intersection=>true,
                           :overapproximation=>OctDirections,
                           :lazy_R⋂I=>false,
                           :lazy_R⋂G=>false,
                           :lazy_A⌜R⋂G⌟⋂I=>false)
    sol = solve(RodReactor, options, opC, opD)
end;

In [ ]:
# using Concrete Discrete Post
@time begin
    opC = BFFPSV18(:δ=>0.01)
    opD = ConcreteDiscretePost()
    sol = solve(RodReactor, options, opC, opD) # does it produce 
end;

In [ ]:
using LazySets.Approximations: project, overapproximate

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 3], Hyperrectangle) for i in eachindex(sol.Xk)];
plot(Xkproj, xlab="x", ylab="c2")

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 2], Hyperrectangle) for i in eachindex(sol.Xk)];
plot(Xkproj, xlab="x", ylab="c1")